In [6]:
import pandas as pd

import tensorflow_datasets as tfds

from tqdm import tqdm



dataset_train = tfds.load("imdb_reviews", split="train",shuffle_files=True)
dataset_test = tfds.load("imdb_reviews", split="test",shuffle_files=True)


In [11]:
def convert_to_df(dataset):
    data = [{"text":item["text"].numpy().decode("utf-8"), "label":item["label"].numpy()} for item in tqdm(dataset)]
    return pd.DataFrame(data)

In [13]:
df_train = convert_to_df(dataset_train)
df_test = convert_to_df(dataset_test)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:01<00:00, 13651.52it/s]

  0%|                                                                                                                                              | 0/25000 [00:00<?, ?it/s]2023-04-20 11:08:57.587913: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-04-20 11:08:57.588229: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]

In [7]:
def sentiment(value):
    if value == 1:
        return "positive"
    return "negative"

<_PrefetchDataset element_spec={'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [34]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = stopwords.words("english")

[nltk_data] Downloading package stopwords to /home/molesz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
from bs4 import BeautifulSoup


def remove_html_tags(text):
    s = BeautifulSoup(text)
    return s.get_text()

In [19]:
df_train["text"] = df_train["text"].apply(remove_html_tags)


In [21]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/molesz/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [25]:
tokenized_reviews =df_train["text"].apply(lambda review_text: word_tokenize(review_text.replace("\n","").lower() ))

In [26]:
tokenized_reviews.head()

0    [this, was, an, absolutely, terrible, movie, ....
1    [i, have, been, known, to, fall, asleep, durin...
2    [mann, photographs, the, alberta, rocky, mount...
3    [this, is, the, kind, of, film, for, a, snowy,...
4    [as, others, have, mentioned, ,, all, the, wom...
Name: text, dtype: object

In [35]:
d = dict()
for review in tqdm(tokenized_reviews):
    for word in review:
        if word not in STOPWORDS  and word.isalpha():
            d[word] = d.get(word,0) + 1


 76%|████████████████████████████████████████████████████████████████████████████████████████████████▉                               | 18926/25000 [00:04<00:01, 3505.47it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:06<00:00, 3991.10it/s]


In [36]:
d = sorted(d.items(), key=lambda item: item[1], reverse=True)
d[:10]

[('movie', 42586),
 ('film', 38763),
 ('one', 25574),
 ('like', 19664),
 ('good', 14612),
 ('would', 13378),
 ('even', 12452),
 ('time', 11904),
 ('really', 11682),
 ('story', 11559)]

In [38]:
DESIRED_VOCAB_SIZE = 4000

VOCAB = [k for k,v in d[:DESIRED_VOCAB_SIZE]]
word_table = pd.DataFrame({"word": VOCAB})
word_table.head(10)

,word
0,movie
1,film
2,one
3,like
4,good
5,would
6,even
7,time
8,really
9,story


In [44]:
dic_freqs = {"positive": {}, "negative": {}}

VOCAB_IDX = {}
for i in range(0, len(word_table["word"].values)):
    VOCAB_IDX[word_table["word"].values[i]] = i
    
for idx in range(df_train.shape[0]):
    review = df_train.iloc[idx]["text"]
    sentiment = df_train.iloc[idx]["sentiment"]
    
    
    for word in review.split(" "):
        if word in VOCAB_IDX:
            dic_freq[sentiment][word] = dic_freq[sentiment].get(word,0) + 1

KeyError: 'sentiment'